# Debugging:
## This script imports C and Python HDF output for comparison

In [433]:
import h5py
import numpy as np
import matplotlib.style
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('classic')
%matplotlib inline
import subprocess

# Labels and timestamps

In [434]:
# and its filename
#
py_filename = "output_acoustic_wave_high_low_mach_gravity_comp.h5"
py_filename = "output_internal_long_wave_low_mach_gravity_comp.h5"
py_filename = "output_travelling_vortex_3d_48_low_mach_gravity_comp.h5"

#####################################################

figsize_tuple = (12,3)

## Converter

Converts all hdf4 output of RKLM C-code to hdf5.

In [435]:
def convert_c(time,base_suffix,typ,tag,flux,debug=False):
    base_folder_name = "output"

    base_path = base_folder_name + base_suffix
    folder_comp = "low_Mach_gravity_comp/"
    folder_psinc = "low_Mach_gravity_psinc/"
    folder_comp = "low_Mach_gravity_comp_perturbed/"
    folder_psinc = "low_Mach_gravity_psinc_perturbed/"
    folder_comp = "low_Mach_gravity_comp_original/"
#     folder_psinc = "low_Mach_gravity_psinc_original/"
    
    if typ == 'PI':
        folder_type = folder_psinc
    if typ == 'COMP':
        folder_type = folder_comp

    # Manually list the folder names for each output
    directories = ['S', 'T', 'Y', 'buoy', 'dT', 'dY', 'dp2_c', 'dp2_nodes', 'dpdim', 'drhoY', 'p', 'p2_c', 'p2_nodes', 'rho', 'rhoY', 'rhoe', 'u', 'v', 'vortz', 'w']
    directories += ['rhou', 'rhov', 'rhow', 'rhs']
    directories += ['buoy']

    # And manually list the file names.
    filenames = ['S', 'T', 'Y', 'buoy', 'dT', 'dY', 'dp2_c', 'dp2_n', 'dpdim', 'drhoY', 'p', 'p2_c', 'p2_n', 'rho', 'rhoY', 'rhoe', 'u', 'v', 'vortz', 'w']
    filenames += ['rhou', 'rhov', 'rhow', 'rhs']
    filenames += ['buoy']

    # specify the format extensions for input and output of converter
    hdf_format = '.'+ 'hdf'
    h5_format = '.' + 'h5'

    if flux == True:
        if tag == 'before_advect' or tag == 'after_half_step':
            directories += ['rhoYu', 'rhoYv']
            filenames += ['rhoYu', 'rhoYv']

    if tag == 'after_ebnaimp' or tag == 'after_full_step':
        directories += ['wplusx', 'wplusy', 'hcenter', 'pnew', 'rhs_nodes', 'p2_initial']
        filenames += ['wplusx', 'wplusy', 'hcenter', 'p2_full', 'rhs_nodes', 'p2_initial']

    string_hdf = '_' + time + hdf_format
    string_h5 = '_' + time + h5_format
    full_path = base_path + folder_type

    lst_hdf = []
    lst_h5 = []

    # build list of paths from parameters specified above.
    i = 0
    for directory in directories:
        lst_hdf.append(full_path + directory + '/' + filenames[i] + string_hdf)
        lst_h5.append(full_path + directory + '/' + filenames[i] + string_h5)
        i += 1 

    # print the list of paths
    if debug == True:
        for path in lst_hdf:
            print(path)

    # now, convert the arrays specified in the list of paths
    for item in lst_hdf:
        p = subprocess.call(["./h4toh5convert", item])
        
    return string_h5, full_path

## Import C hdf5 output

Import the test case arrays from the c-output.

In [436]:
def import_c(string_h5, full_path, flux, tag):
    # manually specific folder names
    test_cases_folders = ['p2_c', 'dp2_nodes', 'p', 'p2_nodes', 'rho', 'rhoY', 'rhoe', 'rhou', 'rhov', 'rhow', 'u', 'vortz', 'dpdim', 'T', 'dT', 'Y', 'dY', 'dp2_c', 'rhs']

    # and filenames
    test_cases_names = ['p2_c', 'dp2_n', 'p', 'p2_n', 'rho', 'rhoY', 'rhoe', 'rhou', 'rhov', 'rhow', 'u', 'vortz', 'dpdim', 'T', 'dT', 'Y', 'dY', 'dp2_c', 'rhs']

    if flux == True:
        if tag == 'before_advect' or tag == 'after_half_step':
            test_cases_folders += ['rhoYu','rhoYv']
            test_cases_names += ['rhoYu', 'rhoYv']

    if tag == 'after_ebnaimp' or tag == 'after_full_step':
        test_cases_folders += ['wplusx', 'wplusy', 'hcenter', 'pnew', 'rhs_nodes', 'p2_initial']
        test_cases_names += ['wplusx', 'wplusy', 'hcenter', 'p2_full', 'rhs_nodes', 'p2_initial']

    test_cases_folders = np.char.array(test_cases_folders)
    test_cases_names = np.char.array(test_cases_names)

    # build paths from folder names and filenames
    test_cases_paths = full_path + test_cases_folders + '/' + test_cases_names + string_h5

    # define empty class as a holder for all the C-hdf5 output.
    class c_output(object):
        def __init__(self):
            None

        def c_out(self):
            for key,value in vars(self).items():
                setattr(self,key,value['Data-Set-2'])

    # get an instance of an empty class to populate it with the C-HDF output as attributes
    c = c_output()

    # populate the class with the C-HDF output as attributes
    i = 0
    for path in test_cases_paths:
        setattr(c,test_cases_folders[i],h5py.File(path, 'r'))
        i += 1


    # run the method to extract array from HDF objects
    c.c_out()
    return c

## Import the Python output

In [437]:
# specify directory where the python hdf5 output is.
# py_directory = "../RKLM_Python/"
py_directory = "../"

py_path = py_directory + py_filename

# helper function to extract array from hdf file by its name and timestep

label = '_' + label
def py_out(py_dataset,time=label):
    return pyfile[str(py_dataset)][str(py_dataset)+time][:]

def get_filename(grid_x,grid_y,size,end_time,suffix):
    fn = "output_travelling_vortex_ensemble=%i_%i_%i_%.1f_%s.h5" %(size,grid_x,grid_y,end_time,suffix)
    return fn
#     return "output_travelling_vortex_ensemble=" + str(size) + "_" + str(grid_x) + "_" + str(grid_y) + "_" + str(end_time) + suffix + ".h5"

def get_path(filename):
    return py_directory + filename

def py_out(pyfile,py_dataset,time):
    return pyfile[str(py_dataset)][str(py_dataset)+time][:]

inner = (slice(2,-2),slice(2,-2))
#     inner = (slice(None,),slice(None,))

def ensemble_test_case(time, path, N, attribute, tag):

    file = h5py.File(path,'r')

    array = []
    for n in range(N):
        
        if type(time) == 'float':
            t_label = '_ensemble_mem=%i_%.3d_after_full_step' %(n,time)
        else:
#             t_label = '_ensemble_mem=%i_%s_after_full_step' %(n,time)
            t_label = '_%.3d_%s' %(time,tag)
        array.append(py_out(file,attribute,time=t_label)[inner])

    array = np.array(array)
    array = array.mean(axis=0)

    file.close()
    file.close()
    return array

def diff_plot(py_arr,c_arr,time,att):
    fig, ax = plt.subplots(ncols=3, figsize=(15,5))
    im0 = ax[0].imshow(c_arr[inner])
    ax[0].set_title("(48x48), C-output")
    im1 = ax[1].imshow(py_arr)
    fig.colorbar(im0,ax=ax[0],shrink=.75)
    ax[1].set_title("(48x48), Py-output")
    fig.colorbar(im1,ax=ax[1],shrink=.75)
    im2 = ax[2].imshow(np.abs(c_arr[inner] - py_arr))
    ax[2].set_title("abs. diff. between outputs")
    fig.colorbar(im2,ax=ax[2],shrink=.75)
    fig.suptitle(r'Comparing measurements at time step=%.3d for quantity %s' %(time,att), fontsize='16')
    plt.show()

In [438]:
py_time = 0

# typ = 'COMP'
typ = 'PI'

tag = 'ic'
# tag = 'before_advect'
# tag = 'after_advect'
# tag = 'after_ebnaexp'
tag = 'after_ebnaimp'
# tag = 'after_half_step'
# tag = 'after_efna'
# tag = 'after_full_advect'
# tag = 'after_full_ebnaexp'
tag = 'after_full_step'

#####################################################
#
# Time labels for C and Python
#
t_label = '%.3d' %py_time
time = t_label + '_' + tag
#####################################################

#####################################################
#
# C-output folder names
#
# base_suffix = '_acoustic_wave_high/'
# base_suffix = '_internal_long_wave/'
# base_suffix = '_travelling_vortex_3d_48_no_initial_projection/'
base_suffix = '_travelling_vortex_3d_48_with_initial_projection/'
# 
#####################################################

py_directory = "../"
py_directory += "output_travelling_vortex/"

flux = False

string_h5, full_path = convert_c(time,base_suffix,typ,flux,tag)
c = import_c(string_h5, full_path,flux,tag)

grid_x = 48
grid_y = 48
ref_size = 1
end_time = 1.0

if typ == 'COMP': suffix = 'comp'
if typ == 'PI': suffix = 'psinc'

py_filename_ref = get_filename(grid_x, grid_y, ref_size, end_time, suffix)
py_path = get_path(py_filename_ref)

atts = ['rho',
          'rhou',
          'rhov',
          'rhoY',
          'p2_nodes',
         ]

if tag == 'after_ebnaimp' or tag == 'after_full_step':
    atts += ['rhs_nodes']

# if tag == 'before_advect' or tag == 'after_half_step':
#     atts += ['rhoYu', 'rhoYv']

# py_arr = ensemble_test_case(py_time,py_path,1,'p2_nodes',tag)
# c_arr = c.p2_nodes

for att in atts:
    py_arr = ensemble_test_case(py_time,py_path,1,att,tag)
    c_arr = getattr(c,att)
    diff_plot(py_arr,c_arr,py_time,att)

# print((np.abs(c.rho[inner] - py_rho) / c.rho[inner]).max())

OSError: Unable to open file (unable to open file: name = 'output_travelling_vortex_3d_48_with_initial_projection/low_Mach_gravity_psinc_perturbed/wplusx/wplusx_000_after_full_step.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)